In [126]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [127]:
def training(len_input, embed_space_size, embed_action_size):
    '''define layer
    embed_space_size: space種類數
    embed_action_size: action種類數
    len_input: (space預計輸入維度, action預計輸入維度, others預計輸入維度)
    '''
    input_space = keras.Input(shape=len_input[0], name='Space_input')
    input_action = keras.Input(shape=len_input[1], name='Action_input')
    input_other = keras.Input(shape=len_input[2], name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    
    layer_reshape_space = layers.Reshape((4,), input_shape=(len_input[0], 4), name='reshape_space')
    layer_reshape_action = layers.Reshape((4,), input_shape=(len_input[1], 4), name='reshape_action')

    input_concat_embedding = layers.Concatenate(name='Input_merging', axis=1)
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_other]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    embeded_space = layer_reshape_space(embeded_space)
    embeded_action = layer_reshape_action(embeded_action)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, input_other])
    output_prob = layer_dense(embeded_input)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

---

prepare data

In [128]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [129]:
train_input = train_df[['Team', 'No.', 'Space', 'Action']]
train_label = train_df[['Erros', 'Score']]

In [130]:
train_space = train_input[['Space']].astype('float')
train_action = train_input[['Action']].astype('float')
train_others = train_input[['Team', 'No.']]

# train_space = train_space.astype({'Space': 'str'})
# train_action = train_action.astype({'Action': 'str'})
train_others = train_others.astype({'No.': 'str'})

In [131]:
# train_space_dummy = pd.get_dummies(train_space)
# train_action_dummy = pd.get_dummies(train_action)
train_others_dummy = pd.get_dummies(train_others).astype('float')

In [132]:
train_label.loc[:, 'Nothing'] = 0.0
train_label.loc[(train_label['Erros'] == 0) & (train_label['Score'] == 0), 'Nothing'] = 1.0

c:\Users\user\anaconda3\envs\SportScience\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\user\anaconda3\envs\SportScience\lib\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
c:\Users\user\anaconda3\envs\SportScience\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a co

In [133]:
print(train_space.shape)
print(train_action.shape)
print(train_others_dummy.shape)

print(train_label.shape)

(900, 1)
(900, 1)
(900, 24)
(900, 3)


In [134]:
# Avoid tensorflow use full memory
physical_devices = tf.config.experimental.list_physical_devices('CPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [135]:
embed_space_size = 15
embed_action_size = 9
model = training(
    (1, 1, 24), 
    embed_space_size, 
    embed_action_size
    )
    
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
Space_embedding (Embedding)     (None, 1, 4)         60          Space_input[0][0]                
__________________________________________________________________________________________________
Action_embedding (Embedding)    (None, 1, 4)         36          Action_input[0][0]               
_________________________________________________________________________________________

In [136]:
# x = [[[1, 20]], [[2, 3]], [[1, 1]]]
# x[1: :]

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Reshape((2,), input_shape=(1, 2)))
# # model.output_shape == (None, 3, 4), `None` is the batch size.
# model.output_shape
# model.input_shape

In [137]:
regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [138]:
train_space_tensor = tf.convert_to_tensor(train_space)
train_action_tensor = tf.convert_to_tensor(train_action)
train_others_tensor = tf.convert_to_tensor(train_others_dummy)
train_label_tensor = tf.convert_to_tensor(train_label)

# train_space_tensor = tf.expand_dims(train_space_tensor, 0)
# train_action_tensor = tf.expand_dims(train_action_tensor, 0)
# train_others_tensor = tf.expand_dims(train_others_tensor, 0)
# train_label_tensor = tf.expand_dims(train_label_tensor, 0)


train_x = [train_space_tensor, train_action_tensor, train_others_tensor]

In [139]:
pred = model.fit(train_x, train_label_tensor, epochs=epochs, callbacks=[callbacks])

Epoch 1/20
29/29 [==============================] - 1s 2ms/step - loss: 1.1276 - auc: 0.5307
Epoch 2/20
29/29 [==============================] - 0s 2ms/step - loss: 1.0572 - auc: 0.6331
Epoch 3/20
29/29 [==============================] - 0s 1ms/step - loss: 0.9910 - auc: 0.7221
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 0.9308 - auc: 0.7827
Epoch 5/20
29/29 [==============================] - 0s 3ms/step - loss: 0.8763 - auc: 0.8177
Epoch 6/20
29/29 [==============================] - 0s 2ms/step - loss: 0.8321 - auc: 0.8333
Epoch 7/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7959 - auc: 0.8484
Epoch 8/20
29/29 [==============================] - 0s 3ms/step - loss: 0.7669 - auc: 0.8586
Epoch 9/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7432 - auc: 0.8655
Epoch 10/20
29/29 [==============================] - 0s 2ms/step - loss: 0.7249 - auc: 0.8703
Epoch 11/20
29/29 [==============================] - 0s 2ms/step - lo